In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
#import outcome file with org_uuid
main_df = pd.read_csv(R"d:/msc-project/data/final/outcome_final.csv",encoding='utf-8',
                         index_col=False,
                         parse_dates= ['founded_on','seed_date','series_a_date','series_b_date','series_c_date'])
main_processing_df = main_df[['org_uuid','name','outcome']]

In [3]:
main_processing_df.head(3)

,org_uuid,name,outcome
0,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,0
1,00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,0
2,000095de-8e2b-82f1-32a7-c222ba3d5682,Ultraprise Loan Technologies,0


In [4]:
len(main_processing_df)

142245

In [5]:
#import jobs file
jobs_df = pd.read_csv(R"d:/msc-project/data/pre-processed/jobs_preprocessed.csv",encoding='utf-8',
                         index_col=False,
                         parse_dates= ['started_on','ended_on'])
jobs_df.head(3)

,job_uuid,job_name,people_uuid,person_name,org_uuid,org_name,started_on,ended_on,is_current,title,job_type,is_founder
0,697b6934-fc1f-9d63-cfb2-1a10759b378e,Ben Elowitz Co-Founder/CEO/Board of Directors ...,ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,Co-Founder/CEO/Board of Directors,executive,1
1,b1de3765-442e-b556-9304-551c2a055901,Kevin Flaherty VP Marketing @ Wetpaint,5ceca97b-493c-1446-6249-5aaa33464763,Kevin Flaherty,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,NaN,False,VP Marketing,executive,0
2,1319cd30-f5e8-c700-0af6-64029c6f7124,Raju Vegesna Chief Evangelist @ Zoho,9f99a98a-aa97-b30b-0d36-db67c1d277e0,Raju Vegesna,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,2000-11-01,NaN,True,Chief Evangelist,employee,0


In [6]:
num_founder = pd.DataFrame(jobs_df.groupby('org_uuid')['is_founder'].sum())
main_processing_df=pd.merge(main_processing_df,
                         num_founder,
                         on='org_uuid',
                         how='left'
                         )
main_processing_df.rename(columns = {'is_founder':'num_founder'},inplace=True)
main_processing_df.head(3)

,org_uuid,name,outcome,num_founder
0,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,0,4.0
1,00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,0,2.0
2,000095de-8e2b-82f1-32a7-c222ba3d5682,Ultraprise Loan Technologies,0,NaN


In [7]:
#drop companies with no founder info
main_processing_df.dropna(subset=['num_founder'],inplace=True)
main_processing_df = main_processing_df[main_processing_df['num_founder'] > 0]

In [8]:
len(main_processing_df)

78693

In [9]:
founder_df = jobs_df[jobs_df['is_founder'] == 1]
founder_gb = founder_df.groupby('org_uuid')

In [10]:
founder_gb.get_group('000014da-0c46-b9cb-0941-3a93c027b119')['people_uuid']

981847    18134829-3fc2-d700-a7f4-2dc95e8882a6
981850    a20b6e51-5f9b-4cb3-9942-d65f8acdcec5
981854    c7ca9967-869d-7775-37c4-a2d5c1569c46
981861    a9606b0f-fbb4-cd04-9c7d-663fd362ec5e
Name: people_uuid, dtype: object

In [11]:
people_df = pd.read_csv(R"d:/msc-project/data/pre-processed/people_preprocessed.csv",encoding='utf-8',
                         index_col=False)

In [12]:
people_df.columns

Index(['people_uuid', 'people_name', 'first_name', 'last_name', 'gender',
       'country_code', 'state_code', 'region', 'city', 'org_uuid',
       'featured_job_organization_name', 'featured_job_title', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'description'],
      dtype='object')

In [13]:
founder_gender_df = pd.merge(founder_df[['people_uuid','org_uuid']],
                             people_df[['people_uuid','gender']],
                             on='people_uuid',
                             how='left')

In [14]:
founder_gender_gb = founder_gender_df.groupby('org_uuid')

In [15]:
gender_diversity = list()

for org_uuid in list(main_processing_df['org_uuid']):
    founders_gender = list(founder_gender_gb.get_group(org_uuid)['gender']) #create list of founder's gender for one organisation
    if 'male' in founders_gender and 'female' in founders_gender: #assign 1 if male and female in gender list
        gender_diversity.append(1)
    else:
        gender_diversity.append(0)
        
main_processing_df['gender_diversity'] = gender_diversity

In [16]:
main_processing_df.shape

(78693, 5)

In [17]:
main_processing_df.head(3)

,org_uuid,name,outcome,num_founder,gender_diversity
0,000014da-0c46-b9cb-0941-3a93c027b119,Resilio,0,4.0,1
1,00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,0,2.0,0
3,0000d497-c93a-eea3-eeb0-a943dfb4f71e,AutoOffer,0,1.0,0


In [18]:
people_df.cl

AttributeError: 'DataFrame' object has no attribute 'cl'